# Demonstrations of improved recovery with regularization, constraints, and modified model parameterization

We show simple examples of Pogit models that demonstrate how regularizers, constraints, and different link functions can improve our estimation of p and lambda

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from regmod.data import Data
from regmod.variable import Variable, SplineVariable
from regmod.prior import GaussianPrior, UniformPrior, SplineUniformPrior, LinearGaussianPrior
from regmod.models import PogitModel
from regmod.utils import SplineSpecs
from regmod.optimizer import scipy_optimize

## Unconstrained Model
Generate data according to `logit(p) = -sin(2\pi x0)` and `log(lambda) = cos(2\pi x1)` with `x0, x1 ~ Uniform(0,1)`. Over many realizations of the data, fit the model and illustrate the poor recovery.

In [ ]:
np.random.seed(123)
num_obs = 500
nTrials = 50

# Set up variables for the model
var0 = SplineVariable(name="x0",
                      spline_specs=SplineSpecs(knots=np.array([0.0, 0.25, 0.75, 1.0]),
                                               knots_type="abs",
                                               degree=3))

var1 = SplineVariable(name="x1",
                      spline_specs=SplineSpecs(knots=np.array([0.0, 0.5, 1.0]),
                                               knots_type="abs",
                                               degree=3))

# Set up x-axis values for plotting the results
x0_pred = np.linspace(0, 1, 100)
x1_pred = np.linspace(0, 1, 100)
df_pred = pd.DataFrame({"x0": x0_pred, "x1": x1_pred})
data_pred = Data(col_covs=["x0", "x1"], df=df_pred)

pred0 = np.zeros((len(x0_pred), nTrials))
pred1 = np.zeros((len(x1_pred), nTrials))

for i in range(nTrials):
    # Generate data
    x0 = np.random.rand(num_obs)
    x1 = np.random.rand(num_obs)

    true_p = 1.0/(1.0 + np.exp(-np.sin(x0*2.0*np.pi)))
    true_lam = 15.0 + np.exp(np.cos(x1*2.0*np.pi))
    n = np.random.poisson(true_lam)
    y = np.random.binomial(n=n, p=true_p)
    
    # Build and fit model
    df = pd.DataFrame({"y": y, "x0": x0, "x1": x1})
    data = Data(col_obs="y", col_covs=["x0", "x1"], df=df)
    model = PogitModel(data, param_specs={"p": {"variables": [var0]}, "lam": {"variables": [var1]}})
    result = scipy_optimize(model)
    
    # Record the estimated pHat and lambdaHat at every value of x0, x1
    coefs = model.split_coefs(result["coefs"])

    pred0[:,i] = model.params[0].get_param(coefs[0], data_pred)
    pred1[:,i] = model.params[1].get_param(coefs[1], data_pred)


In [ ]:
# Get the mean pHat and lamHat estimate, as well as a 95% confidence interval on the estimates, from the nTrials
# Compute the mean in transformed space, otherwise lambda will be drawn up by its outliers
alpha = 0.05

predMu = np.sort(pred0 * pred1, axis=1)

pred0 = np.sort(pred0, axis=1)
pred1 = np.sort(pred1, axis=1)

def logit(p):
    return np.log(p/(1-p))

def expit(x):
    return np.exp(x)/(1+np.exp(x))

pred0_mean = expit(np.mean(logit(pred0), axis=1))
pred1_mean = np.exp(np.mean(np.log(pred1), axis=1))
predMu_mean = np.exp(np.mean(np.log(predMu), axis=1))

pred0_lcb = pred0[:, int(alpha*nTrials)]
pred1_lcb = pred1[:, int(alpha*nTrials)]
predMu_lcb = predMu[:, int(alpha*nTrials)]

pred0_ucb = pred0[:, int((1-alpha)*nTrials)]
pred1_ucb = pred1[:, int((1-alpha)*nTrials)]
predMu_ucb = predMu[:, int((1-alpha)*nTrials)]

In [ ]:
# Plot the mean pHat and lamHat estimate, as well as a 95% confidence interval on the estimates, from the nTrials
alpha = 0.05

fig, ax = plt.subplots(3, 1, figsize=(10, 5*3))
ax[0].scatter(x0, y/n, marker=".", color="gray", alpha=0.5, s=3)
ax[0].plot(x0_pred, pred0_mean, color="#008080", label="model")
ax[0].plot(x0_pred, 1.0/(1.0 + np.exp(-np.sin(x0_pred*2.0*np.pi))), color="#DC143C", label="true", linestyle="--")
ax[0].fill_between(x0_pred, pred0_lcb, pred0_ucb, color='#008080', alpha=0.2)
ax[0].set_xlabel("x0")
ax[0].set_ylabel("p ~ y/n")
ax[0].set_title("Pogit Model", loc="left")
ax[0].legend()

ax[1].scatter(x1, n, marker=".", color="gray", alpha=0.5, s=3)
ax[1].plot(x1_pred, pred1_mean, color="#008080", label="model")
ax[1].plot(x1_pred, 15.0 + np.exp(np.cos(x1_pred*2.0*np.pi)), color="#DC143C", label="true", linestyle="--")
ax[1].fill_between(x1_pred, pred1_lcb, pred1_ucb, color='#008080', alpha=0.2)
ax[1].set_xlabel("x1")
ax[1].set_ylabel("lam ~ n")
ax[1].legend()

ax[2].scatter(x0, y, marker=".", color="gray")
ax[2].plot(x0_pred, predMu_mean, color="#008080", label="model")
ax[2].plot(x0_pred, (15.0 + np.exp(np.cos(x1_pred*2.0*np.pi)))/(1.0 + np.exp(-np.sin(x0_pred*2.0*np.pi))), color="#DC143C", label="true", linestyle="--")
ax[2].fill_between(x0_pred, predMu_lcb, predMu_ucb, color='#008080', alpha=0.2)
ax[2].set_xlabel("x0, x1")
ax[2].set_ylabel("p*lam ~ y")
ax[2].legend()
#plt.savefig("pogit.pdf", bbox_inches="tight")

plt.show()

## Technique 1: Quadratic Regularizers (Gaussian Priors)

Suppose we knew the average p was about 1/2 - if we regularize toward that, we can improve our recovery of p and lambda

In [ ]:
pred0 = np.zeros((len(x0_pred), nTrials))
pred1 = np.zeros((len(x1_pred), nTrials))

for i in range(nTrials):
    # Generate data
    x0 = np.random.rand(num_obs)
    x1 = np.random.rand(num_obs)

    true_p = 1.0/(1.0 + np.exp(-np.sin(x0*2.0*np.pi)))
    true_lam = 15.0 + np.exp(np.cos(x1*2.0*np.pi))
    n = np.random.poisson(true_lam)
    y = np.random.binomial(n=n, p=true_p)
    
    # Build a new x0 variable that regularizes p toward 1/2
    avgX0 = np.mean(x0)
    avgP_guess = 0.5
    sd = 0.05  # This sd is in logit space; the logit of 0.5 is 0
    design_mat = var0.spline.design_mat(avgX0)
    gaussian_prior_avgP = LinearGaussianPrior(mat=design_mat, mean=logit(np.array(avgP_guess)), sd=sd)

    var0 = SplineVariable(name="x0",
                          spline_specs=SplineSpecs(knots=np.array([0.0, 0.25, 0.75, 1.0]),
                                                   knots_type="abs",
                                                   degree=3),
                         linear_gpriors=[gaussian_prior_avgP])
    
    # Build and fit model WITH REGULARIZTION OF P TOWARD 1/2
    df = pd.DataFrame({"y": y, "x0": x0, "x1": x1})
    data = Data(col_obs="y", col_covs=["x0", "x1"], df=df)
    model = PogitModel(data, param_specs={"p": {"variables": [var0]}, "lam": {"variables": [var1]}})
    result = scipy_optimize(model)
    
    # Record the estimated pHat and lambdaHat at every value of x0, x1
    coefs = model.split_coefs(result["coefs"])

    pred0[:,i] = model.params[0].get_param(coefs[0], data_pred)
    pred1[:,i] = model.params[1].get_param(coefs[1], data_pred)

In [ ]:
# Get the mean pHat and lamHat estimate, as well as a 95% confidence interval on the estimates, from the nTrials
# Compute the mean in transformed space, otherwise lambda will be drawn up by its outliers
alpha = 0.05

predMu = np.sort(pred0 * pred1, axis=1)

pred0 = np.sort(pred0, axis=1)
pred1 = np.sort(pred1, axis=1)

def logit(p):
    return np.log(p/(1-p))

def expit(x):
    return np.exp(x)/(1+np.exp(x))

pred0_mean = expit(np.mean(logit(pred0), axis=1))
pred1_mean = np.exp(np.mean(np.log(pred1), axis=1))
predMu_mean = np.exp(np.mean(np.log(predMu), axis=1))

pred0_lcb = pred0[:, int(alpha*nTrials)]
pred1_lcb = pred1[:, int(alpha*nTrials)]
predMu_lcb = predMu[:, int(alpha*nTrials)]

pred0_ucb = pred0[:, int((1-alpha)*nTrials)]
pred1_ucb = pred1[:, int((1-alpha)*nTrials)]
predMu_ucb = predMu[:, int((1-alpha)*nTrials)]

In [ ]:
# Plot the mean pHat and lamHat estimate, as well as a 95% confidence interval on the estimates, from the nTrials
alpha = 0.05

fig, ax = plt.subplots(3, 1, figsize=(10, 5*3))
ax[0].scatter(x0, y/n, marker=".", color="gray", alpha=0.5, s=3)
ax[0].plot(x0_pred, pred0_mean, color="#008080", label="model")
ax[0].plot(x0_pred, 1.0/(1.0 + np.exp(-np.sin(x0_pred*2.0*np.pi))), color="#DC143C", label="true", linestyle="--")
ax[0].fill_between(x0_pred, pred0_lcb, pred0_ucb, color='#008080', alpha=0.2)
ax[0].set_xlabel("x0")
ax[0].set_ylabel("p ~ y/n")
ax[0].set_title("Pogit Model", loc="left")
ax[0].legend()

ax[1].scatter(x1, n, marker=".", color="gray", alpha=0.5, s=3)
ax[1].plot(x1_pred, pred1_mean, color="#008080", label="model")
ax[1].plot(x1_pred, 15.0 + np.exp(np.cos(x1_pred*2.0*np.pi)), color="#DC143C", label="true", linestyle="--")
ax[1].fill_between(x1_pred, pred1_lcb, pred1_ucb, color='#008080', alpha=0.2)
ax[1].set_xlabel("x1")
ax[1].set_ylabel("lam ~ n")
ax[1].legend()

ax[2].scatter(x0, y, marker=".", color="gray")
ax[2].plot(x0_pred, predMu_mean, color="#008080", label="model")
ax[2].plot(x0_pred, (15.0 + np.exp(np.cos(x1_pred*2.0*np.pi)))/(1.0 + np.exp(-np.sin(x0_pred*2.0*np.pi))), color="#DC143C", label="true", linestyle="--")
ax[2].fill_between(x0_pred, predMu_lcb, predMu_ucb, color='#008080', alpha=0.2)
ax[2].set_xlabel("x0, x1")
ax[2].set_ylabel("p*lam ~ y")
ax[2].legend()
#plt.savefig("pogit.pdf", bbox_inches="tight")

plt.show()

## Technique 2: Linear Constraints

We force lambda to be fit as a convex function of x1. Note that this is only an approximation of the truth; in reality lambda is close to convex, but not quite.

In [ ]:
pred0 = np.zeros((len(x0_pred), nTrials))
pred1 = np.zeros((len(x1_pred), nTrials))

for i in range(nTrials):
    # Generate data
    x0 = np.random.rand(num_obs)
    x1 = np.random.rand(num_obs)

    true_p = 1.0/(1.0 + np.exp(-np.sin(x0*2.0*np.pi)))
    true_lam = 15.0 + np.exp(np.cos(x1*2.0*np.pi))
    n = np.random.poisson(true_lam)
    y = np.random.binomial(n=n, p=true_p)
    
    # Build a new x0 variable that enforces the convexity constraint
    uniform_prior_avgP_cvx = SplineUniformPrior(domain_lb=0,
                                            domain_ub=1,
                                            size=50,
                                            lb=0.0,
                                            ub=np.inf,
                                            domain_type="abs",
                                            order=2)

    var0 = SplineVariable(name="x0",
                          spline_specs=SplineSpecs(knots=np.array([0.0, 0.25, 0.75, 1.0]),
                                                   knots_type="abs",
                                                   degree=3),
                          linear_upriors=[uniform_prior_avgP_cvx])
    
    # Build and fit model WITH CONVEXITY CONSTRAINT IN LAMBDA
    df = pd.DataFrame({"y": y, "x0": x0, "x1": x1})
    data = Data(col_obs="y", col_covs=["x0", "x1"], df=df)
    model = PogitModel(data, param_specs={"p": {"variables": [var0]}, "lam": {"variables": [var1]}})
    result = scipy_optimize(model)
    
    # Record the estimated pHat and lambdaHat at every value of x0, x1
    coefs = model.split_coefs(result["coefs"])

    pred0[:,i] = model.params[0].get_param(coefs[0], data_pred)
    pred1[:,i] = model.params[1].get_param(coefs[1], data_pred)

In [ ]:
# Get the mean pHat and lamHat estimate, as well as a 95% confidence interval on the estimates, from the nTrials
# Compute the mean in transformed space, otherwise lambda will be drawn up by its outliers
alpha = 0.05

predMu = np.sort(pred0 * pred1, axis=1)

pred0 = np.sort(pred0, axis=1)
pred1 = np.sort(pred1, axis=1)

def logit(p):
    return np.log(p/(1-p))

def expit(x):
    return np.exp(x)/(1+np.exp(x))

pred0_mean = expit(np.mean(logit(pred0), axis=1))
pred1_mean = np.exp(np.mean(np.log(pred1), axis=1))
predMu_mean = np.exp(np.mean(np.log(predMu), axis=1))

pred0_lcb = pred0[:, int(alpha*nTrials)]
pred1_lcb = pred1[:, int(alpha*nTrials)]
predMu_lcb = predMu[:, int(alpha*nTrials)]

pred0_ucb = pred0[:, int((1-alpha)*nTrials)]
pred1_ucb = pred1[:, int((1-alpha)*nTrials)]
predMu_ucb = predMu[:, int((1-alpha)*nTrials)]

In [ ]:
# Plot the mean pHat and lamHat estimate, as well as a 95% confidence interval on the estimates, from the nTrials
alpha = 0.05

fig, ax = plt.subplots(3, 1, figsize=(10, 5*3))
ax[0].scatter(x0, y/n, marker=".", color="gray", alpha=0.5, s=3)
ax[0].plot(x0_pred, pred0_mean, color="#008080", label="model")
ax[0].plot(x0_pred, 1.0/(1.0 + np.exp(-np.sin(x0_pred*2.0*np.pi))), color="#DC143C", label="true", linestyle="--")
ax[0].fill_between(x0_pred, pred0_lcb, pred0_ucb, color='#008080', alpha=0.2)
ax[0].set_xlabel("x0")
ax[0].set_ylabel("p ~ y/n")
ax[0].set_title("Pogit Model", loc="left")
ax[0].legend()

ax[1].scatter(x1, n, marker=".", color="gray", alpha=0.5, s=3)
ax[1].plot(x1_pred, pred1_mean, color="#008080", label="model")
ax[1].plot(x1_pred, 15.0 + np.exp(np.cos(x1_pred*2.0*np.pi)), color="#DC143C", label="true", linestyle="--")
ax[1].fill_between(x1_pred, pred1_lcb, pred1_ucb, color='#008080', alpha=0.2)
ax[1].set_xlabel("x1")
ax[1].set_ylabel("lam ~ n")
ax[1].legend()

ax[2].scatter(x0, y, marker=".", color="gray")
ax[2].plot(x0_pred, predMu_mean, color="#008080", label="model")
ax[2].plot(x0_pred, (15.0 + np.exp(np.cos(x1_pred*2.0*np.pi)))/(1.0 + np.exp(-np.sin(x0_pred*2.0*np.pi))), color="#DC143C", label="true", linestyle="--")
ax[2].fill_between(x0_pred, predMu_lcb, predMu_ucb, color='#008080', alpha=0.2)
ax[2].set_xlabel("x0, x1")
ax[2].set_ylabel("p*lam ~ y")
ax[2].legend()
#plt.savefig("pogit.pdf", bbox_inches="tight")

plt.show()

# Technique 3: Modify the Link Function

Suppose we knew that `p` only took values between `0.2` and `0.8`. We modify the inverse link function to be

`l(x) = 0.2 + (0.8 - 0.2)/(1 + exp(-x)`

In [ ]:
from regmod.function import SmoothFunction

def create_texpit(a, b):
    """Create truncated expit function"""
    def texpit_fun(x, a=a, b=b):
        return a + (b-a)/(1 + np.exp(-1*x))
    
    def texpit_dfun(x, a=a, b=b):
        return np.exp(-1*x) * (b-a) / (np.exp(-1*x) + 1)**2
        
    def texpit_d2fun(x, a=a, b=b):
        return np.exp(x) * (np.exp(x) - 1) * (a-b) / (1 + np.exp(x))**3
        
    return SmoothFunction(name="texpit", fun=texpit_fun, dfun=texpit_dfun, d2fun=texpit_d2fun)

In [ ]:
np.random.seed(42)
pred0 = np.zeros((len(x0_pred), nTrials))
pred1 = np.zeros((len(x1_pred), nTrials))

texpit = create_texpit(0.2, 0.8)

for i in range(nTrials):
    # Generate data
    x0 = np.random.rand(num_obs)
    x1 = np.random.rand(num_obs)

    true_p = 1.0/(1.0 + np.exp(-np.sin(x0*2.0*np.pi)))
    true_lam = 15.0 + np.exp(np.cos(x1*2.0*np.pi))
    n = np.random.poisson(true_lam)
    y = np.random.binomial(n=n, p=true_p)
    
    var0 = SplineVariable(name="x0",
                          spline_specs=SplineSpecs(knots=np.array([0.0, 0.25, 0.75, 1.0]),
                                                   knots_type="abs",
                                                   degree=3))
    
    # Build and fit model WITH CONVEXITY CONSTRAINT IN LAMBDA
    df = pd.DataFrame({"y": y, "x0": x0, "x1": x1})
    data = Data(col_obs="y", col_covs=["x0", "x1"], df=df)
    model = PogitModel(data, param_specs={"p": {"inv_link": texpit, "variables": [var0]}, "lam": {"variables": [var1]}})
    result = scipy_optimize(model)
    
    # Record the estimated pHat and lambdaHat at every value of x0, x1
    coefs = model.split_coefs(result["coefs"])

    pred0[:,i] = model.params[0].get_param(coefs[0], data_pred)
    pred1[:,i] = model.params[1].get_param(coefs[1], data_pred)

In [ ]:
# Get the mean pHat and lamHat estimate, as well as a 95% confidence interval on the estimates, from the nTrials
# Compute the mean in transformed space, otherwise lambda will be drawn up by its outliers
alpha = 0.05

predMu = np.sort(pred0 * pred1, axis=1)

pred0 = np.sort(pred0, axis=1)
pred1 = np.sort(pred1, axis=1)

def logit(p):
    return np.log(p/(1-p))

def expit(x):
    return np.exp(x)/(1+np.exp(x))

pred0_mean = expit(np.mean(logit(pred0), axis=1))
pred1_mean = np.exp(np.mean(np.log(pred1), axis=1))
predMu_mean = np.exp(np.mean(np.log(predMu), axis=1))

pred0_lcb = pred0[:, int(alpha*nTrials)]
pred1_lcb = pred1[:, int(alpha*nTrials)]
predMu_lcb = predMu[:, int(alpha*nTrials)]

pred0_ucb = pred0[:, int((1-alpha)*nTrials)]
pred1_ucb = pred1[:, int((1-alpha)*nTrials)]
predMu_ucb = predMu[:, int((1-alpha)*nTrials)]

In [ ]:
# Plot the mean pHat and lamHat estimate, as well as a 95% confidence interval on the estimates, from the nTrials
fig, ax = plt.subplots(3, 1, figsize=(10, 5*3))
ax[0].scatter(x0, y/n, marker=".", color="gray", alpha=0.5, s=3)
ax[0].plot(x0_pred, pred0_mean, color="#008080", label="model")
ax[0].plot(x0_pred, 1.0/(1.0 + np.exp(-np.sin(x0_pred*2.0*np.pi))), color="#DC143C", label="true", linestyle="--")
ax[0].fill_between(x0_pred, pred0_lcb, pred0_ucb, color='#008080', alpha=0.2)
ax[0].set_xlabel("x0")
ax[0].set_ylabel("p ~ y/n")
ax[0].set_title("Pogit Model", loc="left")
ax[0].legend()

ax[1].scatter(x1, n, marker=".", color="gray", alpha=0.5, s=3)
ax[1].plot(x1_pred, pred1_mean, color="#008080", label="model")
ax[1].plot(x1_pred, 15.0 + np.exp(np.cos(x1_pred*2.0*np.pi)), color="#DC143C", label="true", linestyle="--")
ax[1].fill_between(x1_pred, pred1_lcb, pred1_ucb, color='#008080', alpha=0.2)
ax[1].set_xlabel("x1")
ax[1].set_ylabel("lam ~ n")
ax[1].legend()

ax[2].scatter(x0, y, marker=".", color="gray")
ax[2].plot(x0_pred, predMu_mean, color="#008080", label="model")
ax[2].plot(x0_pred, (15.0 + np.exp(np.cos(x1_pred*2.0*np.pi)))/(1.0 + np.exp(-np.sin(x0_pred*2.0*np.pi))), color="#DC143C", label="true", linestyle="--")
ax[2].fill_between(x0_pred, predMu_lcb, predMu_ucb, color='#008080', alpha=0.2)
ax[2].set_xlabel("x0, x1")
ax[2].set_ylabel("p*lam ~ y")
ax[2].legend()
#plt.savefig("pogit.pdf", bbox_inches="tight")

plt.show()